In [9]:
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy

In [10]:
image_asset_path = "./images/"
# test_image = './TEST/TEST2.png'
test_image = './images/N1.jpeg'
test_video = './TEST/TEST.mov'
yolo_model  = '../yolov5/runs/train/Model2/weights/best.onnx'

# settings
INPUT_WIDTH =  640
INPUT_HEIGHT = 640

# LOAD THE IMAGE
img = io.imread(test_image)

fig = px.imshow(img)
fig.update_layout(width=700, height=400, margin=dict(l=10, r=10, b=10, t=10))
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()

In [11]:
# LOAD YOLO MODEL
# net = cv2.dnn.readNetFromONNX(yolo_model)
# net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
# net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load the YOLO model
net = cv2.dnn.readNetFromONNX(yolo_model)

The next step is I need to parse this image to my YOLO model and get the prediction. Important - size we need to set as input width a 640 and input height is 640.

We have the 25200 rows and 6 columns.  The first four columns will give you the information about the bounding box, which is center X and, Center Y, W and H. And these values are normalized to 640 by 640. And the next two values are the confidence and the probabilities. Confidence tells us what is the confidence for the detecting the bounding box and the next one is the probability score of the class, since we have only one class that is number plate.

Let’s make a list of boxes and confidence and place it order. As well we need to identify our X_factor and Y_factor of image.

We have to filter detections based on the confidence score and the probability score. Let me take the confidence >0.4 and filter from class >0.25.

There is will be one of major issue with YOLO modeling it will give you the repeated boxes, but the same object it might give you that multiple boxes. For that we need to perform the non-maximum suppression. To do it we need to provide all our clean boxes and confidences. 

In the end of that part of code I will take bounding boxes and draw the rectangle box to our image. And will write it all in functions. 

Here we will not only detect license plate but else extract test from it. We already done it before let’s repeat it again. We will write function. First from bounding box we need to take our x,y,w,h and extract ROI. I place it at top of our functions in drawing part.

In [12]:
INPUT_WIDTH, INPUT_HEIGHT = 416, 416

def extract_text(image, bbox):
    x,y,w,h = bbox
    roi = image[y:y+h, x:x+w]

    # show roi image
    fig = px.imshow(roi)
    fig.update_layout(width=300, height=200, margin=dict(l=10, r=10, b=10, t=10))
    fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
    fig.show()

    if 0 in roi.shape:
        print('no number')
        return 'no number'
    else:
        text = pt.image_to_string(roi)
        text = text.strip()
        print(text)
        return text

def get_detections(img,net):
    image = img.copy()
    row, col, d = image.shape
    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image

    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH,INPUT_HEIGHT),swapRB=True,crop=False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]

    return input_image, detections

def non_maximum_supression(input_image,detections):
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w/INPUT_WIDTH
    y_factor = image_h/INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0.4:
            class_score = row[5]
            if class_score > 0.25:
                cx, cy , w, h = row[0:4]

                left = int((cx - 0.5*w)*x_factor)
                top = int((cy-0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box = np.array([left,top,width,height])

                confidences.append(confidence)
                boxes.append(box)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)

    return boxes_np, confidences_np, index

def drawings(image,boxes_np,confidences_np,index):
    license_texts = []
    for ind in index:
        x,y,w,h =  boxes_np[ind[0]]
        bb_conf = confidences_np[ind[0]]
        conf_text = 'plate: {:.0f}%'.format(bb_conf*100)
        license_text = extract_text(image,[x,y,w,h])

        license_texts.append(license_text)

        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
        cv2.rectangle(image,(x,y+h),(x+w,y+h+25),(0,0,0),-1)

        cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
        cv2.putText(image,license_text,(x,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)

    return image, license_texts

def yolo_predictions(img,net):
    input_image, detections = get_detections(img,net)
    boxes_np, confidences_np, index = non_maximum_supression(input_image, detections)
    result_img, license_texts = drawings(img,boxes_np,confidences_np,index)
    return result_img, license_texts


In [14]:
# test
test_image = './images/N2.jpeg'
img = io.imread(test_image)
results = yolo_predictions(img,net)

fig = px.imshow(img)
fig.update_layout(width=700, height=400, margin=dict(l=10, r=10, b=10, t=10))
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\layers\reshape_layer.cpp:109: error: (-215:Assertion failed) total(srcShape, srcRange.start, srcRange.end) == maskTotal in function 'cv::dnn::computeShapeByReshapeMask'


In [ ]:
# Image paths
image_paths = {
    image_asset_path + 'N2.png': ['42-UK-32'], # license plates in image1.png
    image_asset_path + 'image2.png': ['DEF456'], # license plates in image2.png
}

# Load the YOLO model
net = cv2.dnn.readNetFromONNX(yolo_model)

# Placeholder for the results
results = {}

# Process each image
for image_path, true_plates in image_paths.items():
    # Load the image
    img = io.imread(image_path)
    
    # Predict the plates
    result_img, detected_plates = yolo_predictions(img, net)
    
    # Store the result
    results[image_path] = {
        'true_plates': true_plates,
        'detected_plates': detected_plates
    }

# Now, 'results' contains the true plates and detected plates for each image

# For accuracy, we will simply count the number of correctly detected plates
correct = 0
total = 0

for image_path, data in results.items():
    correct += len(set(data['true_plates']) & set(data['detected_plates']))
    total += len(data['true_plates'])

accuracy = correct / total if total > 0 else 0

print('Accuracy:', accuracy)
